# Generate force coefficients

The following steps are performed to obtain all the artifacts for force coefficient use case:

1. Read parameters file
2. Read Mesh description
   * Its path can be specified
   * Or it can be obtained from the simulation configuration file
3. Read data for pressure coefficient

Instantiate path manager

In [4]:
from cfdmod.use_cases.pressure.path_manager import CfPathManager
import pathlib

path_manager = CfPathManager(output_path=pathlib.Path("./output/pressure"))

Read parameters file

In [5]:
from cfdmod.use_cases.pressure.force.Cf_config import CfCaseConfig

config_path=pathlib.Path("./fixtures/tests/pressure/Cf_params.yaml")
post_proc_cfg = CfCaseConfig.from_file(config_path)

post_proc_cfg.force_coefficient

{'measurement_1': CfConfig(bodies=['building', 'building2'], variables=['Cfx', 'Cfy', 'Cfz'], statistics=['min', 'max', 'std', 'avg', 'skewness', 'kurtosis']),
 'measurement_2': CfConfig(bodies=['marquise', 'lanternim'], variables=['Cfx', 'Cfy', 'Cfz'], statistics=['min', 'max', 'std', 'avg', 'skewness', 'kurtosis'])}

Read Mesh from file

In [6]:
from nassu.lnas import LagrangianFormat

mesh_path=pathlib.Path("./fixtures/tests/pressure/galpao/galpao.normalized.lnas")
mesh = LagrangianFormat.from_file(mesh_path)

mesh.name, mesh.geometry.vertices.shape, mesh.geometry.triangles.shape

('galpao', (1549, 3), (2915, 3))

Read pressure coefficient data

In [7]:
import pandas as pd

cp_data_path=pathlib.Path("./fixtures/tests/pressure/data/cp_t.resampled.hdf")
cp_data = pd.read_hdf(cp_data_path)

cp_data_to_use = (
    cp_data.to_frame() if isinstance(cp_data, pd.Series) else cp_data
)
cp_data_to_use

,time_step,point_idx,cp
0,10000.0,0,0.268273
1,10000.0,1,0.273546
2,10000.0,2,0.265319
3,10000.0,3,0.276913
4,10000.0,4,0.272593
...,...,...,...
294410,10100.0,2910,0.485344
294411,10100.0,2911,0.474767
294412,10100.0,2912,0.450817
294413,10100.0,2913,0.537118


Add areas to pressure coefficient data

In [8]:
vec_areas = mesh.geometry._cross_prod() / 2
areas_df = pd.DataFrame({"Ax": vec_areas[:, 0], "Ay": vec_areas[:, 1], "Az": vec_areas[:, 2]})
areas_df["point_idx"] = areas_df.index

cp_data = pd.merge(cp_data, areas_df,
    on="point_idx",
    how="left"
)

Calculate force coefficient

In [9]:
from cfdmod.use_cases.pressure.force.Cf_data import process_body

for cfg_label, cfg in post_proc_cfg.force_coefficient.items():
    for body_label in cfg.bodies:
        body_cfg = post_proc_cfg.bodies[body_label]
        processed_body = process_body(mesh=mesh, body_cfg=body_cfg, cp_data=cp_data, cfg=cfg)
        processed_body.save_outputs(body_label=body_label, cfg_label=cfg_label, path_manager=path_manager)
